# Real Time Energy Data For Low Carbon Training

## Goal

- Select a training location with the lowest real time carbon intensity to perform low carbon training

## Basic Process

1. Retrieve real time carbon intensity data by API

2. Select the region with the lowest carbon intensity for training 

3. Perform low carbon training

## Detail Process

1. Set up GCP

    - Account info

    - Project info, i.e. project ID

    - Credentials

2. Initialize Vertex AI

    - `from google.cloud import aiplatform`
    
    - Initialize with project ID and credentials

3. Set up training environment

    - Import environment

    - Load environment

    - Import request

    - Import helper

    - Import json

4. Load the Electricity Maps API
    - [Free personal tier API](https://api-portal.electricitymaps.com/)

5. Select the training location with the real time lowest carbon intensity


    - Method 1: Define carbon intensity function with selected training location

        - Define an url by getting latitude and longitude on Google Map

        - Define get request

        - Load json with API returned content

            - Carbon intensity

            - Datetime

            - UpdatedAt time

            - CreatedAt time

            - Renewable energy type and its amount
            
            - Power consumption amount
    
    - Method 2: Define cleanest function with region list

        - Return the training location with the real time lowest carbon intensity out of region list

6. Define task.py as a training job

    - Import libraries

    - Create/ load a dataset
    
    - Create model

7. Define storage bucket

    - `from google.cloud import storage`

    - Define with project ID and credentials

8. Define custom training job

    - Include selected training location

    - Include task.py training job

    - Include storage bucket
    
    - Include display job name

9. Train model in selected training location

10. Delete bucket

## Out of Scope

- Carbon footprint tracking report per project and cloud account

## Remarks 

- One alternative way is to define Power_stats to view both real time carbon intensity and power breakdown data of a selected training location in one step

In [ ]:
import helper, requests, json, numpy as np
def power_stats(lat,lon, api_key=helper.load_emaps_api_key()):
    coordinates = { "lat": lat, "lon": lon }

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    breakdown_abridged = {
        'renewablePercentage': breakdown['renewablePercentage'],
        'fossilFreePercentage': breakdown['fossilFreePercentage'],
        'powerConsumptionBreakdown': breakdown['powerConsumptionBreakdown'],
        'consumption_percent': {
            k: np.round((v/breakdown['powerConsumptionTotal']) * 100) 
            for k, v 
            in breakdown['powerConsumptionBreakdown'].items()
        },
    }
    
    return intensity, breakdown_abridged